# Analisi spotify

## Preparazione dei dati per ML

In [1]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline


In [2]:
import pandas as pd

# Carica il dataset pulito
df = pd.read_csv('spotify_clean.csv')

# Controllo rapido
print(df.shape)
df.head()

FileNotFoundError: [Errno 2] No such file or directory: 'spotify_clean.csv'

In [ ]:
# Selezione delle feature
features_num = ['danceability','energy','loudness','tempo','instrumentalness','duration_s']
features_cat = ['key','mode','genre'] if 'genre' in df.columns else ['key','mode']

X = df[features_num + features_cat]
y = df['popularity']

In [ ]:
# Split train/test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Preprocessing: scaling per numeriche + one-hot per categorical
preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), features_num),
        ('cat', OneHotEncoder(drop='first', sparse_output=False), features_cat)
    ])

In [ ]:
df.head()

,track_name,artist_name,release_date,popularity,danceability,energy,loudness,tempo,instrumentalness,key,mode,release_year,release_month,duration_s
0,Song 0,Artist 0,2015-01-01,61.01005,0.328912,0.951039,-11.654537,106.63049,0.055283,8,0,2015,1,265.503
1,Song 1,Artist 1,2025-12-31,61.01005,0.516030,0.789873,-5.013845,181.61456,0.002502,11,0,2025,12,189.023
2,Song 2,Artist 2,2015-01-01,61.01005,0.874080,0.462060,-3.434210,95.62363,0.060388,8,1,2015,1,189.941
3,Song 3,Artist 3,2015-01-01,61.01005,0.600241,0.641216,-1.451914,183.15892,0.014532,2,1,2015,1,271.677
4,Song 4,Artist 4,2015-01-01,61.01005,0.559638,0.837364,-4.470684,121.04664,0.079854,3,1,2015,1,226.054


In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
import pandas as pd

# Carica il dataset pulito
df = pd.read_csv('spotify_clean.csv')
print(df.shape)
df.head()

# Selezione delle feature
features_num = ['danceability','energy','loudness','tempo','instrumentalness','duration_s']
features_cat = ['key','mode','genre'] if 'genre' in df.columns else ['key','mode']

X = df[features_num + features_cat]
y = df['popularity']

# Split train/test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Preprocessing: scaling per numeriche + one-hot per categorical
preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), features_num),
        ('cat', OneHotEncoder(drop='first', sparse_output=False), features_cat)  # qui usiamo sparse=False
    ]
)

# Applica il preprocessing (fit sul train)
X_train_proc = preprocessor.fit_transform(X_train)
X_test_proc = preprocessor.transform(X_test)

print(X_train_proc.shape, X_test_proc.shape)


(1500, 14)
(1200, 18) (300, 18)


In [ ]:
# DEFINIZIONE TARGET E FEATURES
df_clean = df.copy()
df_clean = df_clean.dropna()

target = 'popularity' 
X = df_clean.drop(columns=[target])
y = df_clean[target]


In [ ]:
from sklearn.ensemble import RandomForestRegressor

print("Addestramento del modello in corso...")
model = RandomForestRegressor(n_estimators=100, random_state=42, n_jobs=-1)
model.fit(X_train, y_train)

Addestramento del modello in corso...


,n_estimators,100
,criterion,'squared_error'
,max_depth,None
,min_samples_split,2
,min_samples_leaf,1
,min_weight_fraction_leaf,0.0
,max_features,1.0
,max_leaf_nodes,None
,min_impurity_decrease,0.0
,bootstrap,True
,oob_score,False


In [ ]:
#VALUTAZIONE
from sklearn.metrics import mean_absolute_error, r2_score
predictions = model.predict(X_test)
mae = mean_absolute_error(y_test, predictions)
r2 = r2_score(y_test, predictions)


print(f"Risultati del Modello:")
print(f"Errore Medio Assoluto (MAE): {mae:.2f}")
print(f"R2 Score (Precisione): {r2:.2f}")


Risultati del Modello:
Errore Medio Assoluto (MAE): 12.54
R2 Score (Precisione): -0.08
